### The original type was:

In [1]:
import os
import pandas as pd

import datetime
import eml_parser

In [13]:
### eml 파일의 content 부분 파싱
# 해당 eml 파일의 header content-type 값에 따라 파싱한다.
# header content-type 값이 multipart인 경우 boundary 값과 빈 행('\n')을 이용해 파싱
# header content-type 값이 multipart가 아닌 경우 빈 행('\n')을 이용해 파싱
def content_parsing(idx, file_path):
    f = open(file_path, 'r', encoding='utf-8' )
    contents = {}
    iscontent = False
    contents['content'] = ''
    while True:
        line = f.readline()
        if not line:
            break
        if '\n' == line:
            iscontent = True
        if iscontent == True:
            contents['content'] += line
                
    return pd.DataFrame(data=contents, index=[idx])

In [16]:
### 인자로 받은 디렉토리(folder)에 있는 파일들 중 선택된 파일들에 대해 파싱한 후 DataFrame으로 저장 ###
def parsing(folder, hex_str):
    file_list = os.listdir(folder)
    
    emls_df = pd.DataFrame()    
    count = 0
    
    for i, file in enumerate(file_list):
        # 원하는 파일이 아니면 제외(원하는 파일 선택)
        # 생각보다 오래걸릴 수 있어서 file[0:2] == '01' 과 같이 더 나눠서 반복해서 실행시키는 것 추천
        if not (file[0:1] == hex_str):  # 각자 시작하는 16진수 선택
            continue
        
        if count%1000 == 0:     # 진행 상태 확인(100개 단위로 file 이름 print)
            print('file name:', file)
        count += 1
        
        ###
        eml_df = pd.DataFrame(data={'file_name':file}, index=[i])
        
        #### content-parsing ####
        file_path = os.path.join(folder, file)
        eml_df = pd.concat([eml_df, content_parsing(i, file_path)], axis=1)
            
        # error가 발생해도 나머지 파일들에 대한 파싱값 저장하기 위해 try-except 사용
        try:
            emls_df = pd.concat([emls_df, eml_df])  
        except Exception as e:
            print('=== Exception:', e, '(during concat) ===')
            print('error file name:', file)
            
    return emls_df

In [17]:
content_df_f = parsing('dataset/eml', 'f')

file name: f0005103-7cfe-4d92-a770-d19e0bf80e6b.eml
file name: f05cc9e4-1ce6-4c52-b6b9-f85fd642bcc8.eml
file name: f0b629c9-111b-448a-a963-f5d38d2dd762.eml


In [18]:
content_df_f

,file_name,content
681573,f0005103-7cfe-4d92-a770-d19e0bf80e6b.eml,\n\n--__.5DA0A.8DE57EF.F4D4CD\nContent-Type: t...
681574,f000979c-129d-44eb-81be-4f417fca9a53.eml,\n\n--_21B.6DA.6D\nContent-Type: text/html;\nC...
681575,f000b997-4b59-4479-98bc-36668955d8fb.eml,\nDQqBmYGahLSEtIFjgWSBRYSqhLSBY4FkgWSBRYFFgUWB...
681576,f000cfcf-9f3b-4994-8939-9b78fa357271.eml,\nPCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RU...
681577,f000f66a-c144-42ba-a6c4-4116c0cab309.eml,\n\n--..DF_5CB14FF.EE42\nContent-Type: text/ht...
...,...,...
684386,f0ffcea1-e080-4d39-80f1-ba6aa8b624d4.eml,\n\n--67_ED3FFA4EC0_FF.19C4BA_\nContent-Type: ...
684387,f0ffd3c9-bc39-4948-b9bf-d33575bbd91a.eml,\nPCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RU...
684388,f0ffdffe-ed6e-427c-a712-9b5d1815a557.eml,\n\n--7CB2_0.3_.FCE\nContent-Type: text/html;\...
684389,f0fff19f-e38d-4523-ba04-c5c53fb24016.eml,\nPCFET0NUWVBFIGh0bWwgUFVCTElDICItLy9XM0MvL0RU...


In [ ]:
content_df_f.to_csv('eml_body_f.csv', encoding='utf-8-sig')